# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114934e+02     1.563229e+00
 * time: 0.0594480037689209
     1     1.046441e+01     8.944547e-01
 * time: 1.292612075805664
     2    -1.124438e+01     1.038812e+00
 * time: 1.3567490577697754
     3    -3.363942e+01     7.814995e-01
 * time: 1.455306053161621
     4    -4.688039e+01     5.538318e-01
 * time: 1.5242161750793457
     5    -5.656975e+01     2.108468e-01
 * time: 1.594208002090454
     6    -5.959057e+01     1.141833e-01
 * time: 1.6606941223144531
     7    -6.074639e+01     5.054385e-02
 * time: 1.7120070457458496
     8    -6.122966e+01     4.586223e-02
 * time: 1.7688000202178955
     9    -6.151768e+01     3.590275e-02
 * time: 1.8234100341796875
    10    -6.175701e+01     2.862701e-02
 * time: 1.8854851722717285
    11    -6.192644e+01     2.446037e-02
 * time: 1.9404029846191406
    12    -6.201985e+01     2.064478e-02
 * time: 1.9937560558319092
    13    -6.208122e+01     1.761441e-02
 * time: 2.0550131797790

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671079
    AtomicLocal         -18.8557727
    AtomicNonlocal      14.8522672
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336827

    total               -62.261666454136
